In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
from torchsummary import summary

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize']

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8]

In [ ]:
!git clone https://github.com/KuoYuChang/MLDL_video_course.git
%cd MLDL_video_course

In [ ]:
from utils.data import gaussian_data, circle_data, xor_data, spiral_data
from utils.plot_tools import plot_play_data, w2plane

### Gaussian dataset

In [ ]:
mean_0 = [-2, -2]
mean_1 = [2, 2]
cov = [[1, 0], [0, 1]]
num = 100

In [ ]:
gaussian = gaussian_data(mean_0, mean_1, cov=cov, num=num)

In [ ]:
half_num = int(num/2)
left_gau = gaussian[:half_num]
right_gau = gaussian[half_num:]
plot_play_data(right_gau, left_gau)

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.optim as optim

In [ ]:
from utils.dataset_torch import playDataset

In [ ]:
batch_size = 16

In [ ]:
gaussian_xy = gaussian[:, 0:2]
gaussian_labels = gaussian[:, 2]

gaussianData = playDataset(gaussian_labels, gaussian_xy)


gaussianLoader = DataLoader(gaussianData, batch_size=batch_size, shuffle=True)

In [ ]:
dataiter = iter(gaussianLoader)
print(next(dataiter))

In [ ]:
# simple linear layer network

class GaussianNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear = nn.Linear(2, 1)
        self.final_acti = nn.Sigmoid()

        # define loss, optimizer
        self.loss = nn.BCELoss()
        self.optimizer = optim.SGD(self.parameters(), lr=0.002, momentum=0.9)

    def forward(self, x):
        x = self.linear(x)
        x = self.final_acti(x)
        return x

    def train_step(self, inputs, labels):
        outputs = self(inputs)
        output_inside = outputs.flatten()
        loss = self.loss(output_inside, labels)
        loss.backward()

        # update weight via gradient
        self.optimizer.step()

        return loss.item()

In [ ]:
gaussianNet = GaussianNet()

In [ ]:
print(summary(gaussianNet, input_size=(2,)))

In [ ]:
num_epoch = 700
currentLoader = gaussianLoader

for epoch in range(num_epoch):
    running_loss = 0.0

    count = 0
    for i, data in enumerate(currentLoader, 0):
        inputs, labels = data

        # zero the parameter gradients
        gaussianNet.optimizer.zero_grad()

        batch_loss = gaussianNet.train_step(inputs, labels)
        running_loss += batch_loss

        count = count + 1
        
    if epoch % 50 == 49:    # print every 2000 mini-batches
        #print("count: ", count)
        print(f'[{epoch + 1}] loss: {running_loss / count:.3f}')
        running_loss = 0.0
print('Finished Training')

In [ ]:
hyper_plane = gaussianNet.linear
hyper_plane.weight, hyper_plane.bias

In [ ]:
def w2plane(weight, bias):
    wx = weight[0]
    wy = weight[1]
    bias = bias[0]
    a = 0
    b = 0
    if_vertical = False
    if abs(wy) < 1e-8:
        if_vertical = True
        a = 1
        b = -bias/wx
    else:
        a = -wx/wy
        b = -bias/wy
    return a, b, if_vertical

In [ ]:
weight = hyper_plane.weight.cpu().detach().numpy()
weight = weight[0]
bias = hyper_plane.bias.cpu().detach().numpy()
a, b, if_vertical = w2plane(weight, bias)

print(a, b, if_vertical)

In [ ]:
plot_play_data(right_gau, left_gau, predict=True, hy_plane=[a, b], if_vertical=if_vertical)

## circleData

In [ ]:
radius = 5
num = 100
noise = 0.001

In [ ]:
circle = circle_data(radius=radius, num=num, noise=noise)

In [ ]:
inside_circle = circle[circle[:, 2] == 1]
outside_circle = circle[circle[:, 2] == 0]

In [ ]:
plot_play_data(inside_circle, outside_circle)

### hand-craft feature transform

In [ ]:
new_circle = np.zeros((num, 3), dtype=np.float32)

new_circle[:, 1] = np.sqrt( np.multiply(circle[:, 0], circle[:, 0]) + np.multiply(circle[:, 1], circle[:, 1]) )
new_circle[:, 0] = np.divide(circle[:, 0], new_circle[:, 1])
new_circle[:, 2] = circle[:, 2]

In [ ]:
# plot
inside_new_circle = new_circle[new_circle[:, 2] == 1]
outside_new_circle = new_circle[new_circle[:, 2] == 0]
plot_play_data(inside_new_circle, outside_new_circle)

### train linear classifier on transformed circle dataset

In [ ]:
batch_size = 16

In [ ]:
circle_labels = new_circle[:, 2]
new_circle_xy = new_circle[:, 0:2]

newCircleData = playDataset(circle_labels, new_circle_xy)


newCircleLoader = DataLoader(newCircleData, batch_size=batch_size, shuffle=True)

In [ ]:
dataiter = iter(newCircleLoader)
print(next(dataiter))

In [ ]:
# linear layer to classify

class CircleNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear = nn.Linear(2, 1)
        self.final_acti = nn.Sigmoid()

        # define loss, optimizer
        self.loss = nn.BCELoss()
        self.optimizer = optim.SGD(self.parameters(), lr=0.001, momentum=0.9)

    def forward(self, x):
        x = self.linear(x)
        x = self.final_acti(x)
        return x

    def train_step(self, inputs, labels):
        outputs = self(inputs)
        output_inside = outputs.flatten()
        loss = self.loss(output_inside, labels)
        loss.backward()

        # update weight via gradient
        self.optimizer.step()

        return loss.item()

In [ ]:
circleNet = CircleNet()

In [ ]:
print(summary(circleNet, input_size=(2,)))

In [ ]:
hyper_plane = circleNet.linear
hyper_plane.weight, hyper_plane.bias

In [ ]:
num_epoch = 1200
currentLoader = newCircleLoader

for epoch in range(num_epoch):
    running_loss = 0.0

    count = 0
    for i, data in enumerate(currentLoader, 0):
        inputs, labels = data

        # zero the parameter gradients
        circleNet.optimizer.zero_grad()

        batch_loss = circleNet.train_step(inputs, labels)
        running_loss += batch_loss

        # if i % 3 == 2:    # print every 2000 mini-batches
            # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            # running_loss = 0.0
        count = count + 1
        
    if epoch % 50 == 49:    # print every 2000 mini-batches
        #print("count: ", count)
        print(f'[{epoch + 1}] loss: {running_loss / count:.3f}')
        running_loss = 0.0
print('Finished Training')

In [ ]:
hyper_plane = circleNet.linear
hyper_plane.weight, hyper_plane.bias

In [ ]:
weight = hyper_plane.weight.cpu().detach().numpy()
weight = weight[0]
bias = hyper_plane.bias.cpu().detach().numpy()
a, b, if_vertical = w2plane(weight, bias)

print(a, b, if_vertical)

In [ ]:
# plot 
plot_play_data(inside_new_circle, outside_new_circle, predict=True, hy_plane=[a, b], if_vertical=if_vertical)

In [ ]:
newCircleFixLoader = DataLoader(newCircleData, batch_size=1, shuffle=False)
fixDataiter = iter(newCircleFixLoader)
data_i, label_i = next(fixDataiter)

data_i, circleNet(data_i)


In [ ]:
def final_class(prob, thres=0.5):
    if prob > 0.5:
        return 1
    else:
        return 0

In [ ]:
prob = circleNet(data_i).item()
pred_cls = final_class(prob)
pred_cls

In [ ]:
circleNet.eval()

for data in newCircleFixLoader:
    data_i, label_i = data

    prob = circleNet(data_i).item()
    pred_cls = final_class(prob)
    print(data_i, label_i, pred_cls, prob)

### XOR dataset

In [ ]:
xor = xor_data(margin=0.7)

In [ ]:
pos_xor = xor[xor[:, 2]==1]
neg_xor = xor[xor[:, 2]==0]

plot_play_data(pos_xor, neg_xor)

### hand-craft feature transform

In [ ]:
new_xor = np.zeros((num, 3), dtype=np.float32)

new_xor[:, 0] = 0
new_xor[:, 1] = np.multiply(xor[:, 0], xor[:, 1])
new_xor[:, 2] = xor[:, 2]

In [ ]:
pos_new_xor = new_xor[new_xor[:, 2]==1]
neg_new_xor = new_xor[new_xor[:, 2]==0]

plot_play_data(pos_new_xor, neg_new_xor)

In [ ]:
xor_labels = new_xor[:, 2]
new_xor_xy = new_xor[:, 0:2]

newXorData = playDataset(xor_labels, new_xor_xy)


newXorLoader = DataLoader(newXorData, batch_size=batch_size, shuffle=True)

### train with linear classifier

In [ ]:
# linear layer to classify

class XorNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear = nn.Linear(2, 1)
        self.final_acti = nn.Sigmoid()

        # define loss, optimizer
        self.loss = nn.BCELoss()
        self.optimizer = optim.SGD(self.parameters(), lr=0.001, momentum=0.9)

    def forward(self, x):
        x = self.linear(x)
        x = self.final_acti(x)
        return x

    def train_step(self, inputs, labels):
        outputs = self(inputs)
        output_inside = outputs.flatten()
        loss = self.loss(output_inside, labels)
        loss.backward()

        # update weight via gradient
        self.optimizer.step()

        return loss.item()

In [ ]:
xorNet = XorNet()

In [ ]:
print(summary(xorNet, input_size=(2,)))

In [ ]:
num_epoch = 1200
currentLoader = newXorLoader

for epoch in range(num_epoch):
    running_loss = 0.0

    count = 0
    for i, data in enumerate(currentLoader, 0):
        inputs, labels = data

        # zero the parameter gradients
        xorNet.optimizer.zero_grad()

        batch_loss = xorNet.train_step(inputs, labels)
        running_loss += batch_loss

        # if i % 3 == 2:    # print every 2000 mini-batches
            # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            # running_loss = 0.0
        count = count + 1
        
    if epoch % 50 == 49:    # print every 2000 mini-batches
        #print("count: ", count)
        print(f'[{epoch + 1}] loss: {running_loss / count:.3f}')
        running_loss = 0.0
print('Finished Training')

In [ ]:
hyper_plane = xorNet.linear
weight = hyper_plane.weight.cpu().detach().numpy()
weight = weight[0]
bias = hyper_plane.bias.cpu().detach().numpy()
a, b, if_vertical = w2plane(weight, bias)

print(a, b, if_vertical)

In [ ]:
plot_play_data(pos_new_xor, neg_new_xor, predict=True, hy_plane=[a, b], if_vertical=if_vertical)

### Spiral

In [ ]:
from utils.data import spiral_data

In [ ]:
num = 100

In [ ]:
spiral = spiral_data(num=num)

In [ ]:
pos_spiral = spiral[spiral[:, 2]==1]
neg_spiral = spiral[spiral[:, 2]==0]

plot_play_data(pos_spiral, neg_spiral)

In [ ]:
new_spiral = np.zeros((num, 3), dtype=np.float32)

new_spiral[:, 0] = np.sqrt( np.multiply(spiral[:, 0], spiral[:, 0]) + np.multiply(spiral[:, 1], spiral[:, 1]) )
new_spiral[:, 1] = np.divide(spiral[:, 0], new_spiral[:, 1]+1e-8)
new_spiral[:, 2] = spiral[:, 2]

In [ ]:
pos_new_spiral = new_spiral[new_spiral[:, 2]==1]
neg_new_spiral = new_spiral[new_spiral[:, 2]==0]

plot_play_data(pos_new_spiral, neg_new_spiral)

# artifical features for circle dataset

#### modifed circle network

In [ ]:
import copy

In [ ]:
class CircleNetMod(nn.Module):
    def __init__(self):
        super().__init__()

        self.relu = torch.nn.ReLU()
        

        self.linear0 = nn.Linear(2, 6)
        self.batch_0 = nn.BatchNorm1d(6)
        self.linear1 = nn.Linear(6, 5)
        self.batch_1 = nn.BatchNorm1d(5)
        self.linear2 = nn.Linear(5, 4)
        self.batch_2 = nn.BatchNorm1d(4)
        self.linear3 = nn.Linear(4, 2)
        self.batch_3 = nn.BatchNorm1d(2)

        
        

        # final layer to classify
        self.linear = nn.Linear(2, 1)
        self.final_acti = nn.Sigmoid()

        # define loss, optimizer
        self.loss = nn.BCELoss()
        self.optimizer = optim.SGD(self.parameters(), lr=0.001, momentum=0.9)

    def forward(self, x):
        x = self.linear0(x)
        x = self.relu(x)
        x = self.batch_0(x)

        x = self.linear1(x)
        x = self.relu(x)
        x = self.batch_1(x)

        x = self.linear2(x)
        x = self.relu(x)
        x = self.batch_2(x)

        x = self.linear3(x)
        x = self.relu(x)
        x = self.batch_3(x)

        new_x = x.clone()
        
        x = self.linear(x)
        x = self.final_acti(x)
        return new_x, x

    def train_step(self, inputs, labels):
        new_fea, outputs = self(inputs)
        output_inside = outputs.flatten()
        loss = self.loss(output_inside, labels)
        loss.backward()

        # update weight via gradient
        self.optimizer.step()

        return loss.item()

In [ ]:
circleNetMod = CircleNetMod()

In [ ]:
print(summary(circleNetMod, input_size=(2,)))

In [ ]:
circle_labels = circle[:, 2]
circle_xy = circle[:, 0:2]

circleData = playDataset(circle_labels, circle_xy)

In [ ]:
circleLoader = DataLoader(circleData, batch_size=batch_size, shuffle=True, drop_last=True)
circleFixLoader = DataLoader(circleData, batch_size=1, shuffle=False)

In [ ]:
#circleNetMod.train()

In [ ]:
num_epoch = 300
currentLoader = circleLoader

loss_list = np.zeros(num_epoch)

for epoch in range(num_epoch):
    running_loss = 0.0

    count = 0
    for i, data in enumerate(currentLoader, 0):
        inputs, labels = data

        # zero the parameter gradients
        circleNetMod.optimizer.zero_grad()

        batch_loss = circleNetMod.train_step(inputs, labels)
        running_loss += batch_loss

        # if i % 3 == 2:    # print every 2000 mini-batches
            # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            # running_loss = 0.0
        count = count + 1
    # plot here
    
    tran_circle = np.zeros((num, 3), dtype=np.float32)
    tran_circle[:, 2] = circle[:, 2]
    
    circleNetVis = copy.deepcopy(circleNetMod)
    circleNetVis.eval()
    count = 0
    for data in circleFixLoader:
        data_j, label_j = data

        new_fea, prob = circleNetVis(data_j)

        #print("new feature: ", new_fea, label_j)

        # new_feature to numpy
        tran_circle[count, 0] = new_fea[0, 0].item()
        tran_circle[count, 1] = new_fea[0, 1].item()

        count += 1


    if epoch % 20 == 19:    # print every 2000 mini-batches
        # get plane
        hyper_plane = circleNetMod.linear
        weight = hyper_plane.weight.cpu().detach().numpy()
        weight = weight[0]
        bias = hyper_plane.bias.cpu().detach().numpy()
        a, b, if_vertical = w2plane(weight, bias)
        
    
        
    
        # plot
        inside_tran_circle = tran_circle[tran_circle[:, 2] == 1]
        outside_tran_circle = tran_circle[tran_circle[:, 2] == 0]
    
        if if_vertical:
            plot_play_data(inside_tran_circle, outside_tran_circle)
        else:
            plot_play_data(inside_tran_circle, outside_tran_circle, predict=True, hy_plane=[a, b])
             
    
        print("hyper plane: ", a, b, if_vertical)
        print(f'[{epoch + 1}] loss: {running_loss / count:.3f}')
    
    loss_list[epoch] = running_loss / count

for j in range(num_epoch):
    print(f'[{j + 1}] loss: {loss_list[j]:.3f}')

print('Finished Training')



In [ ]:
circleNetMod.eval()

for data in circleFixLoader:
    data_i, label_i = data

    new_feat, prob = circleNetMod(data_i)
    prob = prob.item()
    pred_cls = final_class(prob)
    print(data_i, label_i, pred_cls, prob)

### references
### * tensorflow playground
### https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.60721&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false

### * github of playground
### https://github.com/tensorflow/playground